In [1]:
# ===============================
# NAVER News Web Crawler _ df2(News Contents) Crawling  
# ===============================

# ---------------------
# * Process 
# ---------------------
# Step0: "Preparation" _ Import Packages 
# Step1: "Setting" 
#     1.1) Set File Directory & Parameters
#     1.2) Functions
# Step2: "Loop" - Find un-processed units & scrap the News 

# ---------------------
# * I/O 
# ---------------------
# Input : 
#     1. "naver_news_crawling_df1.csv" (df1): Contains [keywords, period, title, press, url]
# Output : 
#     1. "naver_news_crawling_df2.csv" (df2): Contains [title, date, contents] 


In [2]:
# ===============================
# Step0: "Preparation" _ Import Packages 
# ===============================
import requests
import time
from itertools import count
import datetime as dt
import time
import glob

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [11]:
# ================================
# Step1: "Setting" 
#     1.1) Set File Directory & Parameters
# ================================

# set file dir. 
code_dir = r'Z:/0_프로젝트/데이터바우처 2020/POC/3팀(박재인PL)/03 유로인스트루먼츠/code/김성근임시/euro_code/' 
data_dir = r'Z:/0_프로젝트/데이터바우처 2020/POC/3팀(박재인PL)/03 유로인스트루먼츠/code/김성근임시/euro_data/'
save_dir = r'Z:/0_프로젝트/데이터바우처 2020/POC/3팀(박재인PL)/03 유로인스트루먼츠/code/김성근임시/euro_data/news_data/' 
logg_dir = r'Z:/0_프로젝트/데이터바우처 2020/POC/3팀(박재인PL)/03 유로인스트루먼츠/code/김성근임시/euro_data/logger/' 

# set browser driver dir.
chrome_dir = data_dir+'chromedriver_win32/chromedriver'

# Today, now 
time_start = dt.datetime.now()

_today=time_start.date().strftime('%Y.%m.%d')
_time_start=time_start.time().strftime('%H:%M:%S')

# Set dir. for file saving
df1_name_head = "naver_news_crawling_df1"
df2_name_head = "naver_news_crawling_df2"

# Assign headless setting 
# headless_mode = True 
# if headless_mode :
#     options = webdriver.ChromeOptions()
#     options.add_argument('headless')    # add 'Headless' 
#     options.add_argument('window-size=1920x1080')    # set current display setting 
#     options.add_argument("disable-gpu") 
#     options.add_argument("lang=ko_KR") 

# Write Log 
verbose_ = False    # Print messages in console 

# Write log in logger.txt
logger_name = '1-02_logger_{0}_{1}.txt'.format(str(_today).replace('.',''), str(_time_start).replace(':',''))   
logg = open(logg_dir+logger_name,'w')

# Assign the End of Process datetime 
_EOP_time = '2020.10.12 08:30:00'
_EOP_ = dt.datetime.strptime(_EOP_time,'%Y.%m.%d %H:%M:%S')


In [12]:
# ================================
#     1.2) Functions
# ================================

# 1.2.1) 
def FindUnprocessedUnits(): 
    """
    뉴스데이터 저장 디렉토리에서 df1 목록과 df2 목록을 비교 후, 
    아직 df2 파일이 생성되지 못한 목록을 DataFrame으로 반환합니다.
    """
    # call every df1.csv file's directories & get unit names 
    df1_dirs = glob.glob(save_dir+df1_name_head+'*.csv')
    df1_ls = list(map(lambda x: x.split('\\'+df1_name_head)[1].replace('.csv',''), df1_dirs))
    
    # call every df2.csv file's directories & get unit names 
    df2_dirs = glob.glob(save_dir+df2_name_head+'*.csv')
#     df2_ls = list(map(lambda x: x.split('\\'+df2_name_head)[1].replace('.csv',''), df2_dirs))
#     df2_ls = list(map(lambda x: x.split('\\'+df2_name_head)[1].split('__')[0], df2_dirs))
    df2_ls = set(map(lambda x: x.split('\\'+df2_name_head)[1].split('__')[0], df2_dirs)) # 중복제거를 위해 set으로 
    
    # get total df1 dir.s & unit names as dataFrame 
    df1_df = pd.DataFrame(data={'df1_dirs': df1_dirs, 'files': df1_ls}) 
    
    # return un-crawled units DataFrame 
    return df1_df.loc[~df1_df['files'].isin(df2_ls), ].reset_index(drop=True)
    

In [13]:
# 1.2.2) 
# 함수 블록 ====================================
# parameter ------
# threshold_ = 1000

""" 함수개선사항 : 
1. 각 URL마다 들어가서 기사 수집 중, '예상치 못한 화면'(기사 호출이 안되서 크롤링에 실패하는 경우)
해당 df1_seg에 대해 '에러 띄우고', '중단'하고, 나머지 df로 계속 수집 수행하기 위한 
try except 추가 

2. 'df1'에 기사 내용이 하나도 없을때, '빈 df2'를 저장해주는 기능 추가 

3. log 기록 삽입 추가 
"""

def DF2Crawler(target_df, threshold_ = 1000):
    """ ====================function from ==========================="""
    # 0. Preparation
    # 개선사항2. 
    empty_df2 = pd.DataFrame(columns=['title','date','content','url'])
    
    for idx, val in target_df.iterrows(): # for 1: '개별df1파일 하나씩 루프 수행'
    #     keyword_ = val[1].split('_')[1]     # ex: 'BGF리테일'
    #     from_ = val[1].split('_')[2].replace('from')     # ex: '2010.01.01'
    #     to_ = val[1].split('_')[3].replace('to')     # ex: '2010.12.31'
    #     print(" * Keyword: {0}, From: {1}, To: {2}") 
    
        artcls_cnt = target_df.shape[0]
        dir_ = val[0] # 'Z:/0_프로젝트/데이터바우처 2020/POC/3팀(박재인PL)/03 유로인스트루먼...	'
        unit_ = val[1] # '_신세계인터내셔날_from2010.01.01_to2010.12.31'

        repo_ = " * Unit: '{0}' ({1}/{2}) ".format(unit_, idx+1, artcls_cnt)
        print(repo_)
        logg.write(repo_+'\n') 
        
        """ 이 안에 한 단위의 크롤러 삽입 """
        """ ====================indentx1 from ==========================="""
        # 단위(unit) 하나 전체 파일 호출 
        df1_whole = pd.read_csv(dir_) # 'Z:/0_프로젝트/데이터.._신세계인터내셔날_to2010.12.31.csv'호출

        # *** length가 0인 경우(기사가 없는 경우), 'continue': 다음 루프 수행
        # 기사가 있다면, 'pass' : 아래 코드 계속 수행 
        # if df1_whole.empty: continue
        if df1_whole.empty: 
            repo_ = '    No articles to scrap for "{0}"'.format(unit_) # '기사없음' 표시
            print(repo_) 
            logg.write(repo_+'\n') 
            save_filename = save_dir+df2_name_head+unit_+'__x.csv' 
            empty_df2.to_csv(save_filename, index=False) # 개선사항2: 빈 df2 저장 
            continue # for 1: 에 다음 루프로 

        # 전체 기사 수 선언
        tot_artc = df1_whole.shape[0]
        print('    The Number of articles:    {0} '.format(tot_artc))

        # threshold_ = 1000 # <<< parameter
        for n, i in enumerate(range(0, tot_artc, threshold_)): # for 2: '기사 1,000개단위로 루프 수행'
            start_ = i 
            end_ = i+threshold_ if i+threshold_<tot_artc else tot_artc
#             print(n, start_, end_)
            """ 1000줄이 넘는 df에 대해서, 해당 .iloc[,] 호출해서 데이터프레임 하나씩 수행 """
            """ ====================indentx2 from ==========================="""
            df1 = df1_whole.iloc[start_:end_,]

            dates=[]
            contents=[]
            df2_urls=[]

            # open blank browser
            driver2=webdriver.Chrome(chrome_dir)

            urls=df1['url'].tolist()
            titles=df1['title'].tolist()
            
            try: # : 개선사항1 try 1: 개별 URL호출 중 끊어진 경우, 다음 df1_seg 루프로 continue
                for url in urls:  # for 3: '개별 기사url 하나씩 루프 수행'
                    # 'urls' from df1: take 'each url' and do crawling
                    try:
                        driver2.get(url)
                        html=driver2.page_source
                        date=driver2.find_element_by_class_name('t11').text
                        #summary=driver.find_element_by_class_name('media_end_summary').text
                        content=driver2.find_element_by_class_name('_article_body_contents').text

                        dates.append(date)
                        contents.append(content)
                        df2_urls.append(url)
                    except: # when 'url' is None, (df1 has no 'None' values from '1-01_v.20.py' )
                        dates.append('NaN')
                        contents.append('NaN')
                        df2_urls.append('NaN')
            except: # : 개선사항1 except 1: 
                repo_ = '     >>> Failed to get News when URL: {0}'.format(url)
                print(repo_) 
                logg.write(repo_+'\n') 
                driver2.close() # 처리되지 못한 웹드라이버 close
                continue # for 2: 에 다음 루프로 

            # Export News Contenst DataFrame For the keyword (For whole period)
            # 데이터 프레임 생성 - 2. 제목, 날짜, 기사 본문
            try : # try 2: 
                mCols2=[]
                df_2=pd.DataFrame(mCols2)

                df_2['title']=titles # 'titles' from df1
                df_2['date']=dates
                df_2['content']=contents
                df_2['url']=df2_urls

                # export news list DataFrame to csv
                save_filename = save_dir+df2_name_head+unit_+'__{0}.csv'.format(n)

#             try : # try 2: <<< 위로 이동
                df_2.to_csv(save_filename, index=False)
                table_report = """    [{0}]_ from'{1}' to '{2}' (df2): successfully Saved  """.format(unit_, start_, end_-1)
                print(table_report)
                logg.write(table_report+'\n') 
            except Exception as e : # except 2: 
                table_report = """    >>> [{0}]_ from'{1}' to '{2}' (df2): failed to export .csv  """.format(unit_, start_, end_-1)
                print(table_report)
                logg.write(table_report+'\n') 

            driver2.close()

            """ ====================indentx2 to ==========================="""


        """ ====================indentx1 to ==========================="""
    """ ====================function to ==========================="""    

In [ ]:
# 3rd run

# ================================
# Step2: "Loop" - Find un-processed units & scrap the News 
# ================================

i_while = 1

while True : 
    # proc1. 미완결 파일 찾기 
    target_df = FindUnprocessedUnits()
    
#         # On-site ===========================================================
#     if target_df.empty: # 모든 파일에 대응하는 df2가 있는 경우 
#         print('\n *** End of Procedures : with {0} times of iteration'.format(i_while-1)) 
#         break # break the loop         
#     elif i_while > 100 : # 루프가 100번 돈 경우
#         break # break the loop         
#     else : # 진행되지 못한 df2가 아직 있는 경우 
#         print('\n  ========= {0} times of iteration: {1} files left to go'.format(i_while, target_df.shape[0]))
        

        # off-site ==========================================================
    if dt.datetime.now() > _EOP_ : # 지정 작업 종료 일시를 지난 경우, 종료 
        print('\n *** End of Procedures : with {0} times of iteration : EoP at {1}'.format(i_while-1, str(dt.datetime.now()))) 
        break # break the loop 
    elif target_df.empty: # 모든 파일에 대응하는 df2가 있는 경우 
        print('\n *** Temporary 1hrs pause : at {0} since there has no valid df1 files'.format(str(dt.datetime.now()))) 
        time.sleep(3600) # 1시간 일시정지 (df1이 생성될 시간을 확보) 
        continue # <<< 1시간 이상 걸리는 df1수집으로 target_df가 생성되지 않았을 경우에 detour 
    else : # 진행되지 못한 df2가 아직 있는 경우 
        print('\n  ========= {0} times of iteration: {1} files left to go'.format(i_while, target_df.shape[0]))
    
    # proc2. 크롤링 수행 
    try: 
        DF2Crawler(target_df, threshold_ = 1000) 
    except: 
        print('\n *** UnexpectedException ocurred: *************************** ')
              
    i_while += 1 # Number of iteration 

# save the log.txt 
logg.close() 


  ========= 1 times of iteration: 757 files left to go
 * Unit: '_AJ렌터카_from2011.01.01_to2011.12.31' (1/757) 
    The Number of articles:    68 
    [_AJ렌터카_from2011.01.01_to2011.12.31]_ from'0' to '67' (df2): successfully Saved  
 * Unit: '_AP위성_from2010.01.01_to2010.12.31' (2/757) 
    The Number of articles:    9 
    [_AP위성_from2010.01.01_to2010.12.31]_ from'0' to '8' (df2): successfully Saved  
 * Unit: '_BGF리테일_from2011.01.01_to2011.12.31' (3/757) 
    No articles to scrap for "_BGF리테일_from2011.01.01_to2011.12.31"
 * Unit: '_BYC_from2011.01.01_to2011.12.31' (4/757) 
    The Number of articles:    87 
    [_BYC_from2011.01.01_to2011.12.31]_ from'0' to '86' (df2): successfully Saved  
 * Unit: '_CJ E&M_from2010.01.01_to2010.12.31' (5/757) 
    The Number of articles:    1601 
    [_CJ E&M_from2010.01.01_to2010.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_CJ E&M_from2010.01.01_to2010.12.31]_ from'1000' to '1600' (df2): successfully Saved  
 * Unit: '_CJ_from2011.01.01

    [_광동제약_from2011.01.01_to2011.12.31]_ from'1000' to '1035' (df2): successfully Saved  
 * Unit: '_광주신세계_from2011.01.01_to2011.12.31' (43/757) 
    The Number of articles:    463 
    [_광주신세계_from2011.01.01_to2011.12.31]_ from'0' to '462' (df2): successfully Saved  
 * Unit: '_국도화학_from2011.01.01_to2011.12.31' (44/757) 
    The Number of articles:    229 
    [_국도화학_from2011.01.01_to2011.12.31]_ from'0' to '228' (df2): successfully Saved  
 * Unit: '_국동_from2011.01.01_to2011.12.31' (45/757) 
    The Number of articles:    192 
    [_국동_from2011.01.01_to2011.12.31]_ from'0' to '191' (df2): successfully Saved  
 * Unit: '_국제약품_from2011.01.01_to2011.12.31' (46/757) 
    The Number of articles:    185 
    [_국제약품_from2011.01.01_to2011.12.31]_ from'0' to '184' (df2): successfully Saved  
 * Unit: '_글로벌에스엠_from2010.01.01_to2010.12.31' (47/757) 
    The Number of articles:    40 
    [_글로벌에스엠_from2010.01.01_to2010.12.31]_ from'0' to '39' (df2): successfully Saved  
 * Unit: '_금양_from2011.01

 * Unit: '_동부라이텍_from2010.01.01_to2010.12.31' (85/757) 
    No articles to scrap for "_동부라이텍_from2010.01.01_to2010.12.31"
 * Unit: '_동부하이텍_from2011.01.01_to2011.12.31' (86/757) 
    The Number of articles:    1205 
    [_동부하이텍_from2011.01.01_to2011.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_동부하이텍_from2011.01.01_to2011.12.31]_ from'1000' to '1204' (df2): successfully Saved  
 * Unit: '_동성코퍼레이션_from2010.01.01_to2010.12.31' (87/757) 
    No articles to scrap for "_동성코퍼레이션_from2010.01.01_to2010.12.31"
 * Unit: '_동아쏘시오홀딩스_from2011.01.01_to2010.12.31' (88/757) 
    No articles to scrap for "_동아쏘시오홀딩스_from2011.01.01_to2010.12.31"
 * Unit: '_동아에스티_from2010.01.01_to2010.12.31' (89/757) 
    No articles to scrap for "_동아에스티_from2010.01.01_to2010.12.31"
 * Unit: '_동아엘텍_from2010.01.01_to2010.12.31' (90/757) 
    The Number of articles:    96 
    [_동아엘텍_from2010.01.01_to2010.12.31]_ from'0' to '95' (df2): successfully Saved  
 * Unit: '_동아지질_from2011.01.01_to2011.12.31' (91/757) 
 

 * Unit: '_롯데정밀화학_from2011.01.01_to2011.12.31' (130/757) 
    No articles to scrap for "_롯데정밀화학_from2011.01.01_to2011.12.31"
 * Unit: '_롯데제과_from2011.01.01_to2011.12.31' (131/757) 
    The Number of articles:    1928 
    [_롯데제과_from2011.01.01_to2011.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_롯데제과_from2011.01.01_to2011.12.31]_ from'1000' to '1927' (df2): successfully Saved  
 * Unit: '_롯데칠성_from2011.01.01_to2011.12.31' (132/757) 
    The Number of articles:    2720 
    [_롯데칠성_from2011.01.01_to2011.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_롯데칠성_from2011.01.01_to2011.12.31]_ from'1000' to '1999' (df2): successfully Saved  
    [_롯데칠성_from2011.01.01_to2011.12.31]_ from'2000' to '2719' (df2): successfully Saved  
 * Unit: '_롯데케미칼_from2011.01.01_to2011.12.31' (133/757) 
    The Number of articles:    6 
    [_롯데케미칼_from2011.01.01_to2011.12.31]_ from'0' to '5' (df2): successfully Saved  
 * Unit: '_롯데푸드_from2011.01.01_to2011.12.31' (134/757) 
    No articles

    [_미래산업_from2011.01.01_to2011.12.31]_ from'2000' to '2092' (df2): successfully Saved  
 * Unit: '_미래테크놀로지_from2010.01.01_to2010.12.31' (171/757) 
    The Number of articles:    6 
    [_미래테크놀로지_from2010.01.01_to2010.12.31]_ from'0' to '5' (df2): successfully Saved  
 * Unit: '_미성포리테크_from2010.01.01_to2010.12.31' (172/757) 
    The Number of articles:    206 
    [_미성포리테크_from2010.01.01_to2010.12.31]_ from'0' to '205' (df2): successfully Saved  
 * Unit: '_미원상사_from2011.01.01_to2011.12.31' (173/757) 
    The Number of articles:    125 
    [_미원상사_from2011.01.01_to2011.12.31]_ from'0' to '124' (df2): successfully Saved  
 * Unit: '_미원홀딩스_from2010.01.01_to2010.12.31' (174/757) 
    No articles to scrap for "_미원홀딩스_from2010.01.01_to2010.12.31"
 * Unit: '_미원화학_from2010.01.01_to2010.12.31' (175/757) 
    The Number of articles:    8 
    [_미원화학_from2010.01.01_to2010.12.31]_ from'0' to '7' (df2): successfully Saved  
 * Unit: '_미코_from2010.01.01_to2010.12.31' (176/757) 
    The Number of a

...

 * Unit: '_삼아알미늄_from2011.01.01_to2011.12.31' (417/1997) 
    The Number of articles:    71 
    [_삼아알미늄_from2011.01.01_to2011.12.31]_ from'0' to '70' (df2): successfully Saved  
 * Unit: '_삼아알미늄_from2012.01.01_to2012.12.31' (418/1997) 
    The Number of articles:    24 
    [_삼아알미늄_from2012.01.01_to2012.12.31]_ from'0' to '23' (df2): successfully Saved  
 * Unit: '_삼아제약_from2011.01.01_to2011.12.31' (419/1997) 
    The Number of articles:    81 
    [_삼아제약_from2011.01.01_to2011.12.31]_ from'0' to '80' (df2): successfully Saved  
 * Unit: '_삼애인더스_from2011.01.01_to2011.12.31' (420/1997) 
    The Number of articles:    2 
    [_삼애인더스_from2011.01.01_to2011.12.31]_ from'0' to '1' (df2): successfully Saved  
 * Unit: '_삼양사_from2011.01.01_to2011.12.31' (421/1997) 
    The Number of articles:    1457 
    [_삼양사_from2011.01.01_to2011.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_삼양사_from2011.01.01_to2011.12.31]_ from'1000' to '1456' (df2): successfully Saved  
 * Unit: '_삼양식품_from2

    [_삼화왕관_from2011.01.01_to2011.12.31]_ from'0' to '143' (df2): successfully Saved  
 * Unit: '_삼화왕관_from2012.01.01_to2012.12.31' (460/1997) 
    The Number of articles:    70 
    [_삼화왕관_from2012.01.01_to2012.12.31]_ from'0' to '69' (df2): successfully Saved  
 * Unit: '_삼화전기_from2011.01.01_to2011.12.31' (461/1997) 
    The Number of articles:    161 
    [_삼화전기_from2011.01.01_to2011.12.31]_ from'0' to '160' (df2): successfully Saved  
 * Unit: '_삼화전기_from2012.01.01_to2012.12.31' (462/1997) 
    The Number of articles:    33 
    [_삼화전기_from2012.01.01_to2012.12.31]_ from'0' to '32' (df2): successfully Saved  
 * Unit: '_삼화전자_from2011.01.01_to2011.12.31' (463/1997) 
    The Number of articles:    102 
    [_삼화전자_from2011.01.01_to2011.12.31]_ from'0' to '101' (df2): successfully Saved  
 * Unit: '_삼화전자_from2012.01.01_to2012.12.31' (464/1997) 
    The Number of articles:    27 
    [_삼화전자_from2012.01.01_to2012.12.31]_ from'0' to '26' (df2): successfully Saved  
 * Unit: '_삼화콘덴서_from2011

    [_서플러스글로벌_from2011.01.01_to2011.12.31]_ from'0' to '2' (df2): successfully Saved  
 * Unit: '_서한_from2011.01.01_to2011.12.31' (500/1997) 
    The Number of articles:    747 
    [_서한_from2011.01.01_to2011.12.31]_ from'0' to '746' (df2): successfully Saved  
 * Unit: '_서호전기_from2011.01.01_to2011.12.31' (501/1997) 
    The Number of articles:    152 
    [_서호전기_from2011.01.01_to2011.12.31]_ from'0' to '151' (df2): successfully Saved  
 * Unit: '_서화정보통신_from2011.01.01_to2011.12.31' (502/1997) 
    The Number of articles:    272 
    [_서화정보통신_from2011.01.01_to2011.12.31]_ from'0' to '271' (df2): successfully Saved  
 * Unit: '_서흥_from2011.01.01_to2011.12.31' (503/1997) 

 *** UnexpectedException ocurred: *************************** 

  ========= 6 times of iteration: 3535 files left to go
 * Unit: '_3R_from2012.01.01_to2012.12.31' (1/3535) 
    The Number of articles:    2082 
    [_3R_from2012.01.01_to2012.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_3R_from2012.01.01_to

 * Unit: '_EG_from2012.01.01_to2012.12.31' (36/3535) 
    The Number of articles:    1799 
    [_EG_from2012.01.01_to2012.12.31]_ from'0' to '999' (df2): successfully Saved  
    [_EG_from2012.01.01_to2012.12.31]_ from'1000' to '1798' (df2): successfully Saved  
 * Unit: '_EMW_from2012.01.01_to2012.12.31' (37/3535) 
    The Number of articles:    53 
    [_EMW_from2012.01.01_to2012.12.31]_ from'0' to '52' (df2): successfully Saved  
 * Unit: '_F&F_from2013.01.01_to2013.12.31' (38/3535) 
    The Number of articles:    280 
    [_F&F_from2013.01.01_to2013.12.31]_ from'0' to '279' (df2): successfully Saved  
 * Unit: '_GBS_from2012.01.01_to2012.12.31' (39/3535) 
    The Number of articles:    44 
    [_GBS_from2012.01.01_to2012.12.31]_ from'0' to '43' (df2): successfully Saved  
 * Unit: '_GH신소재_from2012.01.01_to2012.12.31' (40/3535) 
    No articles to scrap for "_GH신소재_from2012.01.01_to2012.12.31"
 * Unit: '_GKL_from2012.01.01_to2012.12.31' (41/3535) 
    The Number of articles:    1412

 * Unit: '_KG모빌리언스_from2012.01.01_to2012.12.31' (78/3535) 
    The Number of articles:    720 
    [_KG모빌리언스_from2012.01.01_to2012.12.31]_ from'0' to '719' (df2): successfully Saved  
 * Unit: '_KG이니시스_from2012.01.01_to2012.12.31' (79/3535) 
    The Number of articles:    740 
    [_KG이니시스_from2012.01.01_to2012.12.31]_ from'0' to '739' (df2): successfully Saved  
 * Unit: '_KH바텍_from2012.01.01_to2012.12.31' (80/3535) 
    The Number of articles:    190 
    [_KH바텍_from2012.01.01_to2012.12.31]_ from'0' to '189' (df2): successfully Saved  
 * Unit: '_KJ프리텍_from2012.01.01_to2012.12.31' (81/3535) 
    The Number of articles:    97 
    [_KJ프리텍_from2012.01.01_to2012.12.31]_ from'0' to '96' (df2): successfully Saved  
 * Unit: '_KMH_from2012.01.01_to2012.12.31' (82/3535) 
    The Number of articles:    158 
    [_KMH_from2012.01.01_to2012.12.31]_ from'0' to '157' (df2): successfully Saved  
 * Unit: '_KMH하이텍_from2012.01.01_to2012.12.31' (83/3535) 
    No articles to scrap for "_KMH하이텍_from20

In [8]:
""" 
동원_ : ' ... 용역동원', '강동원..' 등 같이 수집됨 
미래산업_ : 
서연_ : 
덕성_ : 
""" 

"\n동원_ : ' ... 용역동원', '강동원..' 등 같이 수집됨 \n미래산업_ : \n서연_ :  \n덕성_ : \n\n"

In [1]:
import sys
sys.pwd

AttributeError: module 'sys' has no attribute 'pwd'